<a href="https://colab.research.google.com/github/valerianXZ/E-Commerce/blob/main/E_commerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time


In [ ]:
from google.colab import files
uploaded=files.upload()

In [3]:
def fetch_data(search_term):

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=options)
    driver.get("https://24h.pchome.com.tw/")
    time.sleep(3)

    search_input = driver.find_element(By.CLASS_NAME, "c-search__input")

    search_input.send_keys(search_term)
    search_input.send_keys(Keys.RETURN)

    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "c-prodInfoV2__title"))
    )

    print("Redirected to:", driver.current_url)

    product_names = driver.find_elements(By.CLASS_NAME, "c-prodInfoV2__title")
    product_prices = driver.find_elements(By.CLASS_NAME, "c-prodInfoV2__priceValue--m")
    product_links = driver.find_elements(By.CSS_SELECTOR, "a.c-prodInfoV2__link.gtmClickV2")

    data = []


    for name, price, link in zip(product_names, product_prices, product_links):
        product_link = link.get_attribute("href")
        data.append({
            'Product': name.text,
            'Price': price.text,
            'Link': product_link
        })
    df = pd.DataFrame(data)
    driver.quit()

    return df


In [4]:
def sort_price(df):
  df['Price'] = df['Price'].replace({'\$': '', ',': ''}, regex=True)
  df['Price']=pd.to_numeric(df['Price'],errors='coerce')
  df=df.sort_values("Price")
  df.loc[df['Price']<4000,'Price']*=12 #for installment prices
  return df

def clean(df):
  df=df.drop_duplicates('Product')
  return df

In [ ]:
product_name="iphone 16 pro"
df=fetch_data(product_name)

if df is not None:
  df=sort_price(df)
  df=clean(df)
  df.to_csv(f"{product_name}_pchome_prices.csv",index=False,encoding='utf-8')
  display(df)





Redirected to: https://24h.pchome.com.tw/search/?q=iphone%2016%20pro


,Product,Price,Link
27,Apple 蘋果 iPhone 16 Pro (128G),24348,https://24h.pchome.com.tw/prod/DYAJFR-1900HSTSY
20,Apple 蘋果 iPhone 16 Pro (256G),26664,https://24h.pchome.com.tw/prod/DYAJFR-1900HSTSJ
11,Apple 蘋果 iPhone 16 Pro Max (256G),29628,https://24h.pchome.com.tw/prod/DYAJFR-1900HSTT4
10,Apple 蘋果 iPhone 16 Pro Max (512G),34248,https://24h.pchome.com.tw/prod/DYAJFR-1900HSTT5
12,Apple 蘋果 【福利機】iPhone 16 Pro 128G 福利品,34100,https://24h.pchome.com.tw/prod/DYAJML-A900IAFT5
38,Apple iPhone 16 Pro Max (256G) -沙漠色鈦金屬,39989,https://24h.pchome.com.tw/prod/DGAV0M-A900I99BW
24,Apple 蘋果 【福利機】iPhone 16 Pro Max 256G 福利品,40888,https://24h.pchome.com.tw/prod/DYAJML-A900IAG5E
39,Apple 蘋果 iPhone 16 Pro (512G),45999,https://24h.pchome.com.tw/prod/DYAJM4-1900HSPQQ
19,Apple 蘋果 iPhone 16 Pro Max (512G) 原廠保固~2025/10/16,46000,https://24h.pchome.com.tw/prod/DYAJMJ-A900I4CBF
17,Apple 蘋果 iPhone 16 Pro (1TB),53399,https://24h.pchome.com.tw/prod/DYAJM8-1900HSPRH
